<div dir="rtl">

# ۲ – ساخت RAG ساده روی اخبار فارسی

در این نوت‌بوک، زیرمجموعه‌ی داده‌ی آماده‌شده را به چند بخش کوچک (چانک) تبدیل می‌کنید، برای هر چانک بردارهای embedding می‌سازید، یک ایندکس برداری (مثلاً با FAISS) ایجاد می‌کنید و در نهایت یک سیستم RAG ساده برای پاسخ‌گویی به سؤال‌های فارسی پیاده‌سازی می‌کنید.

</div>


In [1]:
!pip -q install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 86.1 MB/s eta 0:00:00:00:0100:01


In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

2025-12-20 20:40:21.753823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766263221.928244      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766263221.986964      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766263222.390453      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766263222.390490      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766263222.390492      55 computation_placer.cc:177] computation placer alr

In [3]:
df = pd.read_csv("/kaggle/input/news-subset/news_subset.csv")
df.head()

,id,title,short_link,service,subgroup,abstract,body,tags,published_datetime,agency_name
0,306989,وزیر علوم خواستار شد: تقویت فرهنگ ایثار و شهاد...,http://isna.ir/xdNytx,استان ها,لرستان,ایسنا/لرستان وزیر علوم، تحقیقات و فناوری خواست...,محمدعلی زلفی‌گل ۲۵ اسفندماه در نشست تجلیل از ر...,"استانی-علمی و آموزشی, دفاع‌مقدس, انقلاب اسلامی...",03/17/2023 09:34:00 AM,ISNA
1,288296,یک فعال سیاسی: ارتباط بدون ریا با مردم، ویژگی ...,http://isna.ir/xdN4c6,استان ها,اصفهان,ایسنا/اصفهان یک فعال سیاسی گفت: ارتباط صمیمی و...,ابوالفضل قربانی در گفت‌وگو با ایسنا در آستانه ...,"استانی-سیاسی, شهید سلیمانی, ابوالفضل قربانی, س...",01/02/2023 12:12:00 PM,ISNA
2,178203,اجرای طرح ضربتی مقابله با احتکار و قاچاق کالا ...,http://fna.ir/1otfoe,جامعه,انتظامی و حوادث,سخنگوی پلیس با اشاره به دستور ویژه فرمانده کل ...,به گزارش خبرنگار انتظامی خبرگزاری فارس، سردار ...,"سردار حاجیان, سخنگوی پلیس, دستگیری محتکران",2022-05-05 12:51:24,FarsNews
3,217401,سرمربی تیم ملی فوتسال ژاپن: هویتمان را حفظ کردیم,http://mehrnews.com/xZHWP,ورزش,فوتبال ایران,سرمربی تیم ملی فوتسال ژاپن پس از نایب قهرمانی ...,به گزارش خبرنگار مهر، تیم ملی فوتسال ایران روز...,"تیم ملی فوتسال ژاپن, تیم ملی فوتسال ایران, فوت...",03/09/2023 12:08:00 PM,MehrNews
4,214455,چاپ کتابی درباره روزگار کوفه در آخرالزمان,http://mehrnews.com/xZGjH,فرهنگ و ادب,کتاب و ادبیات,کتاب «پایتخت ملک؛ جستارهایی در ارتباط با کوفه ...,به گزارش خبرگزاری مهر، کتاب «پایتخت ملک؛ جستار...,"شهر کوفه, آخرالزمان, انتشارات شهید کاظمی, مسجد...",03/04/2023 15:12:00 PM,MehrNews


In [4]:
def chunk_text(text, chunk_size=800, overlap=150):
    text = "" if pd.isna(text) else str(text).strip()
    if not text:
        return []
    step = max(1, chunk_size - overlap)
    return [text[i:i + chunk_size] for i in range(0, len(text), step)]

rows = []
chunk_id = 0

for i, r in df.iterrows():
    doc_id = r["id"] if "id" in df.columns else i

    title = r["title"] if "title" in df.columns else ""
    desc = r["body"] if "body" in df.columns else (r["abstract"] if "abstract" in df.columns else "")

    title = "" if pd.isna(title) else str(title).strip()
    desc  = "" if pd.isna(desc)  else str(desc).strip()

    full_text = (title + "\n" + desc).strip() if title else desc
    if not full_text:
        continue

    category = r["subgroup"] if "subgroup" in df.columns else (r["category"] if "category" in df.columns else None)
    date = r["published_datetime"] if "published_datetime" in df.columns else (r["date"] if "date" in df.columns else None)

    for ch in chunk_text(full_text, chunk_size=800, overlap=150):
        rows.append({
            "chunk_id": chunk_id,
            "doc_id": doc_id,
            "text": ch,
            "category": category,
            "date": date,
        })
        chunk_id += 1

chunks_df = pd.DataFrame(rows)
chunks_df.head()
out_path = "/content/chunks_df.csv"
chunks_df.to_csv(out_path, index=False)
print("saved:", out_path)

saved: /content/chunks_df.csv


In [5]:
texts = chunks_df["text"].fillna("").astype(str).tolist()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

semantic_model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    device=device
)

semantic_emb = semantic_model.encode(
    texts,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=False,
).astype(np.float32)

faiss.normalize_L2(semantic_emb)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1115 [00:00<?, ?it/s]

In [9]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=120000,
)
X_tfidf = tfidf.fit_transform(texts)

svd = TruncatedSVD(n_components=256, random_state=42)
lexical_emb = svd.fit_transform(X_tfidf).astype(np.float32)

faiss.normalize_L2(lexical_emb)

In [10]:
hybrid_emb = np.hstack([semantic_emb, lexical_emb]).astype(np.float32)
faiss.normalize_L2(hybrid_emb)

embeddings_semantic = semantic_emb
embeddings_lexical = lexical_emb
embeddings_hybrid = hybrid_emb

embeddings_semantic.shape, embeddings_lexical.shape, embeddings_hybrid.shape

((71326, 768), (71326, 256), (71326, 1024))

In [11]:
# semantic
d_sem = embeddings_semantic.shape[1]
index_semantic = faiss.IndexFlatIP(d_sem)
index_semantic.add(embeddings_semantic)

# lexical
d_lex = embeddings_lexical.shape[1]
index_lexical = faiss.IndexFlatIP(d_lex)
index_lexical.add(embeddings_lexical)

# hybrid
d_hyb = embeddings_hybrid.shape[1]
index_hybrid = faiss.IndexFlatIP(d_hyb)
index_hybrid.add(embeddings_hybrid)

print("semantic:", index_semantic.ntotal, "dim:", d_sem)
print("lexical :", index_lexical.ntotal, "dim:", d_lex)
print("hybrid  :", index_hybrid.ntotal, "dim:", d_hyb)

semantic: 71326 dim: 768
lexical : 71326 dim: 256
hybrid  : 71326 dim: 1024


In [12]:
def retrieve(question, top_k=5, embedding_model="semantic"):
    question = str(question).strip()

    if embedding_model == "semantic":
        q_emb = semantic_model.encode(
            [f"query: {question}"],
            convert_to_numpy=True,
            normalize_embeddings=False
        ).astype(np.float32)
        faiss.normalize_L2(q_emb)
        D, I = index_semantic.search(q_emb, top_k)

    elif embedding_model == "lexical":
        q_tfidf = tfidf.transform([question])
        q_emb = svd.transform(q_tfidf).astype(np.float32)
        faiss.normalize_L2(q_emb)
        D, I = index_lexical.search(q_emb, top_k)

    elif embedding_model == "hybrid":
        q_sem = semantic_model.encode(
            [f"query: {question}"],
            convert_to_numpy=True,
            normalize_embeddings=False
        ).astype(np.float32)
        faiss.normalize_L2(q_sem)

        q_tfidf = tfidf.transform([question])
        q_lex = svd.transform(q_tfidf).astype(np.float32)
        faiss.normalize_L2(q_lex)

        q_emb = np.hstack([q_sem, q_lex]).astype(np.float32)
        faiss.normalize_L2(q_emb)
        D, I = index_hybrid.search(q_emb, top_k)

    else:
        raise ValueError("embedding_model must be one of: 'semantic', 'lexical', 'hybrid'")

    top_chunks = chunks_df.iloc[I[0]][["chunk_id", "doc_id", "text", "category", "date"]].copy()
    top_chunks["score"] = D[0]
    return top_chunks


In [13]:
def answer(question, top_k=5, embedding_model="semantic"):
    retrieved = retrieve(question, top_k=top_k, embedding_model=embedding_model)
    context = "\n\n---\n\n".join(retrieved["text"].astype(str).tolist())
    return context


In [14]:
sample_questions = [
    "آخرین خبرهای مربوط به مذاکرات هسته‌ای چیست؟",
    "در مورد قیمت دلار و وضعیت بازار ارز چه گفته شده؟",
    "نتایج آخرین بازی تیم ملی فوتبال چه بود؟",
    "چه خبرهایی درباره بورس تهران منتشر شده است؟",
    "وضعیت آلودگی هوا در تهران چگونه است؟",
]

for q in sample_questions:
    print("=" * 80)
    print("Q:", q)

    print("\n[Semantic]")
    print(answer(q, top_k=3, embedding_model="semantic")[:1200])

    print("\n[Lexical]")
    print(answer(q, top_k=3, embedding_model="lexical")[:1200])

    print("\n[Hybrid]")
    print(answer(q, top_k=3, embedding_model="hybrid")[:1200])


Q: آخرین خبرهای مربوط به مذاکرات هسته‌ای چیست؟

[Semantic]
غریب آبادی: فراتر از برجام، تعهدی را در حوزه هسته‌ای نمی‌پذیریم
به گزارش خبرنگار گروه سیاست خارجی خبرگزاری فارس، کاظم غریب آبادی سفیر و نماینده دائم جمهوری اسلامی ایران نزد سازمان‌های بین‌المللی در وین امشب (سه‌شنبه) در گفت‌وگویی درباره مذاکرات جاری در وین گفت: مذاکرات وارد موضوعات مهم و اساسی در دو حوزه رفع تحریم‌ و هسته‌ای شده است. این چند موضوع باید حتما حل و فصل شود، هم برای ما از منظر رفع تحریمی و هم از منظر اقدامات هسته‌ای که اگر چنانچه برجام احیا شود باید انجام دهیم. وی که با شبکه خبر گفت‌وگو می‌کرد، افزود: چنانچه این موضوعات حل و فصل نشود، طبیعتا تفاهمی هم شکل نخواهد گرفت. نماینده دائم ایران در آژانس بین‌المللی انرژی اتمی همچنین گفت: این موضوعات تحت بررسی است و این امکان وجود دارد که هیأت‌ها دوباره به پایتخت‌ها برگردند و مشورت‌ها انجام و دستورالعمل‌های لازم اخذ شود. این دیپلمات ارش

---

ین‌المللی انرژی اتمی ۲۱ اسفند با بیان اینکه این طور نیست که بگویید یک شبه به توافق سال ۲۰۱۵ برمی‌گردیم، گفت: از آن زمان تاکنون اتفاق‌ه

In [15]:
llm_id = "Qwen/Qwen2.5-3B-Instruct"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(llm_id)
model = AutoModelForCausalLM.from_pretrained(
    llm_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None,
)

if not torch.cuda.is_available():
    model.to(device)

model.eval()


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((2048,), eps=1e-06)
    (ro

In [16]:
def build_prompt(question: str, retrieved_df: pd.DataFrame) -> str:
    ctx_parts = []
    for _, r in retrieved_df.iterrows():
        ctx_parts.append(
            f"[chunk_id={r['chunk_id']} | doc_id={r['doc_id']} | category={r.get('category', None)} | date={r.get('date', None)}]\n"
            f"{r['text']}"
        )
    context = "\n\n---\n\n".join(ctx_parts)

    prompt = f"""
You are a careful Persian QA assistant.

RULES:
- Answer the question in Persian.
- Use ONLY the information in the retrieved passages below.
- Do NOT add outside knowledge.
- If the passages do not contain enough information to answer, say exactly:
  "اطلاعات کافی در متن‌های بازیابی‌شده نیست."
- When possible, quote short phrases from the passages as evidence.
- Keep the answer concise (3-6 sentences).

Question (Persian):
{question}

Retrieved passages:
{context}

Write the final answer in Persian:
""".strip()
    return prompt


In [17]:
def rag_answer(question, top_k=5, embedding_model="hybrid", max_new_tokens=200):
    retrieved = retrieve(question, top_k=top_k, embedding_model=embedding_model)
    prompt = build_prompt(question, retrieved)

    messages = [
        {"role": "system", "content": "به فارسی و دقیق جواب بده."},
        {"role": "user", "content": prompt},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to(device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
        )

    gen_ids = output_ids[0][input_ids.shape[-1]:]
    answer_text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
    return answer_text, retrieved

In [18]:
ans, ctx = rag_answer("آخرین خبرهای مربوط به مذاکرات هسته‌ای چیست؟", top_k=5, embedding_model="semantic")
print(ans)
ctx.head()

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


آخرین خبرهای مربوط به مذاکرات هسته‌ای، مرتبط با مذاکراتی که ایران و آژانس بین‌المللی انرژی اتمی درباره مباحث هسته‌ای و منطقه‌ای و موشکی می‌خوانند، نشان‌دهنده توافقی برای ادامه مذاکرات است. این توافق باعث شده است که آژانس بین‌المللی انرژی اتمی می‌تواند اطلاعات بیشتری درباره فعالیت‌های هسته‌ای ایران در زمینه آب سنگین، روتورها، سانتریفیوژها و غنی‌سازی ادام


,chunk_id,doc_id,text,category,date,score
46269,46269,81820,غریب آبادی: فراتر از برجام، تعهدی را در حوزه ه...,سیاست خارجی,2021-06-01 10:12:49,0.785847
53546,53546,49179,ین‌المللی انرژی اتمی ۲۱ اسفند با بیان اینکه ای...,سیاست خارجی,2021-03-31 09:04:38,0.740769
52835,52835,133154,و آنها المان‌های جدیدی را در اختیار خواهند دا...,آمریکا، اروپا,2021-09-12 10:30:43,0.738700
52836,52836,133154,ین مذاکرات سازنده بود. به نظر می رسد که با ای...,آمریکا، اروپا,2021-09-12 10:30:43,0.738065
60873,60873,16581,زمشان برای مذاکره مجدد بر سر مباحث هسته ای، مس...,دیگر رسانه ها,2021-01-30 11:33:02,0.735668
